# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

<b>Tasks:</b>
* Predict churn of customers
* Predict likelihood to upgrade
* Predict the lifetime value of the customer based on the above

In [1]:
# import libraries
import os
import pandas as pd
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport

In [2]:
os.listdir()

['README.md',
 'mini_sparkify_event_data.json',
 'Sparkify.ipynb',
 '.gitignore',
 '.ipynb_checkpoints',
 '.git',
 'EDA_report.html']

In [3]:
def load_user_data():
    df = pd.read_json('mini_sparkify_event_data.json',lines=True)
    # convert column names to lowercase
    df.columns = [col.lower() for col in df.columns]
    # remove rows not connected to any users
    df = df.loc[df['userid'].notna()]
    # Remove empty user id records.
    df = df[~(df['userid']=="")]
    return df
    
def generate_eda_report(df):
    # generate EDA report (only if it doesen't exist)
    if os.path.exists('./EDA_report.html'):
        print("EDA report already in the folder.")
    else:
        print("Generating EDA report...")
        profile = ProfileReport(df, title="Pandas Profiling Report")
        profile.to_file("EDA_report.html")
        print("EDA report saved.")

def convert_dates(df,date_columns):
    for col in date_columns:
        df[col] = pd.to_datetime(df[col],unit='ms')
        
def get_dates_range(df):
    date_min = df['ts'].min()
    date_max = df['ts'].max()
    return date_min, date_max

In [4]:
df = load_user_data()

generate_eda_report(df)

convert_dates(df,date_columns=['ts','registration'])

date_min, date_max= get_dates_range(df)
print(f"Dataset interactions range from {date_min} to {date_max}.")

EDA report already in the folder.
Dataset interactions range from 2018-10-01 00:01:57 to 2018-12-03 01:11:16.


In [5]:
df['page'].value_counts()

NextSong                     228108
Thumbs Up                     12551
Home                          10082
Add to Playlist                6526
Add Friend                     4277
Roll Advert                    3933
Logout                         3226
Thumbs Down                    2546
Downgrade                      2055
Settings                       1514
Help                           1454
Upgrade                         499
About                           495
Save Settings                   310
Error                           252
Submit Upgrade                  159
Submit Downgrade                 63
Cancellation Confirmation        52
Cancel                           52
Name: page, dtype: int64

---
# Interaction based Feature Engineering ideas:
* number of songs played (total/last week, month, 3 months )
* number of thumbs Up,thumbs down total thumbs up/total songs played
* number of friends added
* number of songs added to a playlist
* days since registration
* days since last interaction
* dates between registration and last interaction

# Target features:
* Submit upgrade
* Submit downgrade
* Cancellation confirmation

In [6]:
df['iteminsession'].value_counts().describe()

count    1311.000000
mean      212.169336
std       485.799893
min         1.000000
25%         2.000000
50%        11.000000
75%       143.000000
max      2941.000000
Name: iteminsession, dtype: float64

In [7]:
# 'userid': [('user_id','max')], if needed for a join

users = df.groupby('userid').agg(
    {'page': [('page_count', 'count')],
     'length': [('avg_session_length','mean')], 
     'registration': [('registration','max')]
    })

users.columns = users.columns.droplevel()

In [8]:
pseudo code: WIP

df.filter(latest_interaction-time_delta(7days,month,3months))
df.groupby(things from feature engineering section)

SyntaxError: invalid syntax (<ipython-input-8-589ecfb1b488>, line 1)

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.

In [ ]:
# create a Spark session